In [25]:
import numpy as np
import pandas as pd

from mongoengine import connect, DoesNotExist
from mongomodels import Commit, FileAction, File, CodeEntityState, Project, VCSSystem, Hunk, Issue, Event, IssueSystem, Tag
from pycoshark.utils import create_mongodb_uri_string, jira_is_resolved_and_fixed, get_commit_graph, heuristic_renames, java_filename_filter, git_tag_filter, get_affected_versions
from datetime import datetime

In [26]:
db_name = 'smartshark'
db_user = 'bledel'
db_password = 'Q6QTZHhF'
db_host = 'mongoshark.informatik.uni-goettingen.de'
db_port = 27017
db_authentication_db = 'smartshark'
db_ssl = False

uri = create_mongodb_uri_string(db_user, db_password, db_host, db_port, db_authentication_db, db_ssl)

connect(db_name, host=uri, alias='default')

projects = ['ant-ivy', 'archiva', 'calcite', 'cayenne', 'commons-bcel', 'commons-beanutils',
            'commons-codec', 'commons-collections', 'commons-compress', 'commons-configuration',
            'commons-dbcp', 'commons-digester', 'commons-io', 'commons-jcs', 'commons-jexl',
            'commons-lang', 'commons-math', 'commons-net', 'commons-scxml', 
            'commons-validator', 'commons-vfs', 'deltaspike', 'eagle', 'giraph', 'gora', 'jspwiki',
            'knox', 'kylin', 'lens', 'mahout', 'manifoldcf','nutch','opennlp','parquet-mr',
            'santuario-java', 'systemml', 'tika', 'wss4j']

homepages = {
    'ant-ivy':'https://ant.apache.org/ivy/',
    'archiva':'https://archiva.apache.org/',
    'calcite':'https://calcite.apache.org/',
    'cayenne':'https://cayenne.apache.org/',
    'commons-bcel':'https://commons.apache.org/bcel/',
    'commons-beanutils':'https://commons.apache.org/beanutils/',
    'commons-codec':'https://commons.apache.org/codec/',
    'commons-collections':'https://commons.apache.org/collections/',
    'commons-compress':'https://commons.apache.org/compress/',
    'commons-configuration':'https://commons.apache.org/configuration/',
    'commons-dbcp':'https://commons.apache.org/dbcp/',
    'commons-digester':'https://commons.apache.org/digester/',
    'commons-io':'https://commons.apache.org/io/',
    'commons-jcs':'https://commons.apache.org/jcs/',
    'commons-jexl':'https://commons.apache.org/jexl/',
    'commons-lang':'https://commons.apache.org/lang/',
    'commons-math':'https://commons.apache.org/math/',
    'commons-net':'https://commons.apache.org/net/',
    'commons-scxml':'https://commons.apache.org/scxml/',
    'commons-validator':'https://commons.apache.org/validator/',
    'commons-vfs':'https://commons.apache.org/vfs/',
    'deltaspike':'https://deltaspike.apache.org/',
    'eagle':'https://eagle.apache.org/',
    'giraph':'https://giraph.apache.org/',
    'gora':'https://gora.apache.org/',
    'jspwiki':'https://jspwiki.apache.org/',
    'knox':'https://knox.apache.org/',
    'kylin':'http://kylin.apache.org/',
    'lens':'https://lens.apache.org/',
    'mahout':'https://mahout.apache.org/',
    'manifoldcf':'https://manifoldcf.apache.org/',
    'nutch':'https://nutch.apache.org/',
    'opennlp':'https://opennlp.apache.org/',
    'parquet-mr':'https://parquet.apache.org/',
    'santuario-java':'https://santuario.apache.org/',
    'systemml':'https://systemml.apache.org/',
    'tika':'https://tika.apache.org/',
    'wss4j':'https://ws.apache.org/wss4j/'
}

releases = {
    'ant-ivy': [{
        'version': '1.4.1',
        'tag': '1.4.1',
        'revision_hash': 'bfc4a460c2c0730023f3cc0a5e59bb965a94cfe4',
        'corrected_revision': '2e90690057aed8157df0451556709fa4f7bbd6f0'
    },
    {
        'version': '2.0.0',
        'tag': '2.0.0',
        'revision_hash': 'd7c9277d38524c5c971a5ac3e2d23b7bc0d1221c'
    },
    {
        'version': '2.1.0',
        'tag': '2.1.0',
        'revision_hash': '14fa21b35d069eb498a2aff784d59b18679d8ee3'
    },
    {
        'version': '2.2.0',
        'tag': '2.2.0',
        'revision_hash': 'd27f2b35ccb22dd242795c787a7010c2feef13b0'
    },
    {
        'version': '2.3.0',
        'tag': '2.3.0',
        'revision_hash': 'a0ad1bfa81c615443b4916e654fa7087c504cd40'
    },
    {
        'version': '2.4.0',
        'tag': '2.4.0',
        'revision_hash': '0b9db35ee7a94a719e538b04122b86cb997f3a17'
    }],
    'archiva': [{
        'version': '1.0',
        'tag': 'archiva-1.0',
        'revision_hash': '34fb315ea16071c903a6a428706f179e478413d3',
        'corrected_revision': '9ca7fd21ad15fa130d9e1e569da1deba3a1bc23d'
    },{
        'version': '1.1',
        'tag': 'archiva-1.1',
        'revision_hash': 'b3e2ecd39c190116624010587d78de9071008516'
    },{
        'version': '1.2',
        'tag': 'archiva-1.2',
        'revision_hash': 'b27c14eae1314e088ff6872c34bb4b6d40b1df1f'
    },{
        'version': '1.3',
        'tag': 'archiva-1.3',
        'revision_hash': 'c1cfcfc5d39cb1087a6d345732512718e6741ff9'
    },{
        'version': '2.0.0',
        'tag': 'archiva-2.0.0',
        'revision_hash': 'e33eddc47b2ab5096ed2e7f285a9d745d33d86b0'
    },{
        'version': '2.1.0',
        'tag': 'archiva-2.1.0',
        'revision_hash': '93eafa089c11117a1f8a1fdd5752b67386d30b76'
    },{
        'version': '2.2.0',
        'tag': 'archiva-2.2.0',
        'revision_hash': 'e5500cce83acba6a3a756b38620c510f3d00263e'
    }],
    'calcite': [{
        'version': '1.0.0',
        'tag': 'calcite-1.0.0-incubating',
        'revision_hash': '2dd83f21784c4366df84a3aaa62a8b7b0e36d443'
    },{
        'version': '1.1.0',
        'tag': 'calcite-1.1.0-incubating',
        'revision_hash': 'f10ea367ff1cc25497f50f149ef0c91b3ae03031'
    },{
        'version': '1.2.0',
        'tag': 'calcite-1.2.0-incubating',
        'revision_hash': 'd60f2aa3aa9ce7cda7c4f6986af5729d50a28679'
    },{
        'version': '1.3.0',
        'tag': 'calcite-1.3.0-incubating',
        'revision_hash': '495f1859f84b41ae70b2099c3d15c696a49a5100'
    },{
        'version': '1.4.0',
        'tag': 'calcite-1.4.0-incubating',
        'revision_hash': '0c0c203daec56c05b6c75fa3896c8af19844df68'
    },{
        'version': '1.5.0',
        'tag': 'calcite-1.5.0',
        'revision_hash': 'ba6e43c6983ca92d8ce32a693776dbe73f19e0dc'
    },{
        'version': '1.6.0',
        'tag': 'calcite-1.6.0',
        'revision_hash': 'c4d346b0a413a1a62e028dd3be40071523203a58'
    },{
        'version': '1.7.0',
        'tag': 'calcite-1.7.0',
        'revision_hash': '8eebfc6d169421509e5d4792a18917fcb6c44efb'
    },{
        'version': '1.8.0',
        'tag': 'calcite-avatica-1.8.0',
        'revision_hash': 'ddfaf079a8a18be63f042069dd56c50dec4d2066'
    },{
        'version': '1.9.0',
        'tag': 'calcite-1.9.0',
        'revision_hash': '08c56b158ffcfcf205a919cc9fff77a692e649f6'
    },{
        'version': '1.10.0',
        'tag': 'calcite-1.10.0',
        'revision_hash': '448dd85832819b4d22682ffe7908d8058da7c778'
    },{
        'version': '1.11.0',
        'tag': 'calcite-1.11.0',
        'revision_hash': 'f8ba670de4e283d1532f288de53fdb67fa4dea67'
    },{
        'version': '1.12.0',
        'tag': 'calcite-1.12.0',
        'revision_hash': 'ea7ace18cdc26027be690b00f1fcc191aaa87e45'
    },{
        'version': '1.13.0',
        'tag': 'calcite-1.13.0',
        'revision_hash': '54b9823e7ca313bf195c19b7d98f1a06b342cf12'
    },{
        'version': '1.14.0',
        'tag': 'calcite-1.14.0',
        'revision_hash': '7426eef84e7b2ca38cfbf0d47ba53a8abd4d9e38'
    },{
        'version': '1.15.0',
        'tag': 'calcite-1.15.0',
        'revision_hash': 'a2aa00e5b276a30e301ecbf0953e2cbf3a51b605'
    }],
    'cayenne': [{
        'version': '3.0.0',
        'tag': '3.0-final',
        'revision_hash': 'fe8710bdc1316f59fdfde66ea82706995316f725'
    },{
        'version': '3.1.0',
        'tag': 'cayenne-parent-3.1',
        'revision_hash': '63cbf3c98f6e4be000542aa7788875da0939da13'
    }],
    'commons-bcel': [{
        'version': '5.0',
        'tag': 'BCEL_5_0',
        'revision_hash': '4f2b59597e8463d85bad90b48c8d876815291f4f',
        'corrected_revision': '35dc5fc1d4ac353d4c40fef826f8534ba2d1974f'
    },{
        'version': '5.1',
        'tag': 'BCEL_5_1',
        'revision_hash': '0fa7513ef6a2ecdd7e14701fed33f8dcdaed0077',
        'corrected_revision': 'a55dd020f08ed55cbb28834d586e81a13e2db7c0'
    },{
        'version': '5.2',
        'tag': 'BCEL_5_2',
        'revision_hash': 'b546dfdaaaee9eec23c4d0bbe1454a2d6edcaca2',
        'corrected_revision': '1aa7e878e3e3c38a03e154a42cc77f00658c9742'
    },{
        'version': '6.0',
        'tag': 'BCEL_6_0',
        'revision_hash': '647c723ba1262e1ffce520524692b366a7fde45a'
    },{
        'version': '6.1',
        'tag': 'BCEL_6_1',
        'revision_hash': 'f70742fde892c2fac4f0862f8f0b00e121f7d16e'
    },{
        'version': '6.2',
        'tag': 'BCEL_6_2',
        'revision_hash': '893d9bbcdbd5ce764db1a38eccd73af150e5d34d'
    }],
    'commons-beanutils': [{
        'version': '1.0',
        'tag': 'BEANUTILS_1_0',
        'revision_hash': '41f492e7eaed0a9d3f32ee02ca8aa284472d526a',
        'corrected_revision': 'd783d4d1c95895cd1e556180c53c88ab14f8af7a'
    },{
        'version': '1.1',
        'tag': 'BEANUTILS_1_1',
        'revision_hash': 'f4dd0a42d9d642dfc231c1290044fafd281c3fa3',
        'corrected_revision': '68557b16b23bbdd6d13d28b966953efbf1cb77c1'
    },{
        'version': '1.2',
        'tag': 'BEANUTILS_1_2',
        'revision_hash': 'a55d81e2b2b47d73c8dbf5e1de7f0394322290bd',
        'corrected_revision': 'e9fb55e779563377644672db318ef4dc79995cbd'
    },{
        'version': '1.3',
        'tag': 'BEANUTILS_1_3',
        'revision_hash': '227e5d2d4dd00ebbdf49004447e016431648d735',
        'corrected_revision': '297bfe3879a06e63096041e9fd59408b831270e6'
    },{
        'version': '1.4',
        'tag': 'BEANUTILS_1_4',
        'revision_hash': 'dc0f63ae7af19f7897e03d746038882c92d63c4a',
        'corrected_revision': '61b3a30c12c3c734f5fe3cf3a75a7924a860d123'
    },{
        'version': '1.5',
        'tag': 'BEANUTILS_1_5',
        'revision_hash': 'b72aa9e97b85714aad2951b9ff4e2be90a2940a7',
        'corrected_revision': '90f514fc48dfd7e84f7ab38edbe30a581703533b'
    },{
        'version': '1.6',
        'tag': 'BEANUTILS_1_6',
        'revision_hash': '2bc65c91d0c8ac5fac03c41625d10ddee6af7f27',
        'corrected_revision': '5a2d26622b9322a84d59bb48fbcfbb4e0f8b93ca'
    },{
        'version': '1.7.0',
        'tag': 'BEANUTILS_1_7_0',
        'revision_hash': 'cbdadd2011983734383a785466cdc16c06f99fe9',
        'corrected_revision': 'fd5bf514ca3ce36e75a5d9595252fc1a45fc52d8'
    },{
        'version': '1.8.0',
        'tag': 'BEANUTILS_1_8_0',
        'revision_hash': '8fb2285efd72d72a359d9012216260a75bec09b7'
    },{
        'version': '1.9.0',
        'tag': 'BEANUTILS_1_9_0',
        'revision_hash': '9c047b74f426e40b28f497c39572abcf48263650'
    }], 
    'commons-codec':[{
        'version': '1.1',
        'tag': 'CODEC_1_1',
        'revision_hash': 'b4e4eee0644a211c209c21a2056b6227bfee8ccc',
        'corrected_revision': '42d9bf996a0b86b54764c48a2b7859ae77f80dd2'
    },{
        'version': '1.2',
        'tag': 'CODEC_1_2',
        'revision_hash': '2c0905a09d5d04db7e572172bf2ead04293477a2',
        'corrected_revision': 'dea4d55afaa6d6afd4d01b192106148bf20b3f8b'
    },{
        'version': '1.3',
        'tag': 'CODEC_1_3',
        'revision_hash': '1af8f98c3bd96501d558bbe980a2a9f7eb188367',
        'corrected_revision': 'dfad8deb12895a12dd5967dbd99e49707fc3d261'
    },{
        'version': '1.4',
        'tag': 'CODEC_1_4',
        'revision_hash': 'f375d169f8e030169d63d9da00bb14ae2da2ac34'
    },{
        'version': '1.5',
        'tag': 'commons-codec-1.5',
        'revision_hash': '2ce6791741d501bb2e332f6c23ed319d9cf69368'
    },{
        'version': '1.6',
        'tag': '1_6',
        'revision_hash': '23f2aa4fb66aa5b026184ed17b6ad070da183794'
    },{
        'version': '1.7',
        'tag': '1.7',
        'revision_hash': '183aa006e80e5c5a4c8c215d104be39cd77f3336'
    },{
        'version': '1.8',
        'tag': '1.8',
        'revision_hash': 'aa988a92ce3317077abd4e8414ef501b382d7fc0'
    },{
        'version': '1.9',
        'tag': '1.9',
        'revision_hash': '52cfc73491e417366c77c226a7b01b40f7677f9a'
    },{
        'version': '1.10',
        'tag': '1.10',
        'revision_hash': 'e9da3d16ae67f2940a0bbdf982ecec19a0481981'
    },{
        'version': '1.11',
        'tag': 'commons-codec-1.11',
        'revision_hash': '9ceef2231549e95f7868266f38b0c7e1a0a40ed4'
    }], 
    'commons-collections':[{
        'version': '1.0',
        'tag': 'collections-1.0',
        'revision_hash': '5e62ebe3c43f62ca5b58d9deac70154991ce0924',
        'corrected_revision': '5a75bf8e7abd6428e1e88bd9f794bf39519c605e'
    },{
        'version': '2.0',
        'tag': 'collections-2.0',
        'revision_hash': '658dea9545701b813d9bebee1171c508392e472c',
        'corrected_revision': '80c60d68df58af5744b1a521cd68777e62d12312'
    },{
        'version': '2.1',
        'tag': 'collections-2.1',
        'revision_hash': '88186e85e517ff14f5f77b92f03a4f0b01939e36',
        'corrected_revision': '7c223490bc5d82a058973a95f02ac3631ac7d1fa'
    },{
        'version': '3.0',
        'tag': 'collections-3.0',
        'revision_hash': 'ebdf7649e49d164d33c98cc6be4fbfacbbfc4220',
        'corrected_revision': 'b730495d6748604b5a10ab8f6bf4302ff710e1f1'
    },{
        'version': '3.1',
        'tag': 'collections-3.1',
        'revision_hash': '326a1c172f5857709299bc77bd73402352214bbf',
        'corrected_revision': 'a472b7f2bd9f185fb386f9fe33170f2c5de87543'
    },{
        'version': '3.2',
        'tag': 'collections-3.2',
        'revision_hash': '8792072a213366605161b2f6e315024711da7eee',
        'corrected_revision': '4b6411235e6953ad65b4cb0e16ff6ad09581247d'
    },{
        'version': '3.3',
        'tag': 'collections-3.3',
        'revision_hash': '1d0b9abb31faf211192c9866e3ec5dfe86c41e1d'
    },{
        'version': '4.0',
        'tag': 'collections-4.0',
        'revision_hash': 'db189926f7415b9866e76cd8123e40c09c1cc67e'
    },{
        'version': '4.1',
        'tag': 'collections-4.1',
        'revision_hash': 'cb157163d7543f942a1391f3ef752ebea1e1b349'
    }],
    'commons-compress': [{
        'version': '1.0',
        'tag': '1.0',
        'revision_hash': '4b5962d0fc3082e456a7dd249fc25e5aa7b1d350'
    },{
        'version': '1.1',
        'tag': 'commons-compress-1.1',
        'revision_hash': '9ee24a3d6a84c7e09abb5a9b4940bc91e2ab23d6'
    },{
        'version': '1.2',
        'tag': 'COMPRESS_1.2',
        'revision_hash': 'dc733d1ec84c72d68d8cbedc8d4d301eaf975cbf'
    },{
        'version': '1.3',
        'tag': 'COMPRESS_1.3',
        'revision_hash': '0874320927ef9427cf5da5973935a1d17201cdf7'
    },{
        'version': '1.4',
        'tag': 'COMPRESS_1.4',
        'revision_hash': '6e63432a2a15c8b9b7327edb5cd141661890e0e9'
    },{
        'version': '1.5',
        'tag': '1.5',
        'revision_hash': '163c4eee17c041cb7c1ccfedd983816038cc1eca'
    },{
        'version': '1.6',
        'tag': '1.6',
        'revision_hash': 'c49e865f4eabd47e264063499e27e12eb549feb1'
    },{
        'version': '1.7',
        'tag': '1.7',
        'revision_hash': '4a4ba6bfdadc792c59a67935b6c4fe6ee8bfd325'
    },{
        'version': '1.8',
        'tag': '1.8',
        'revision_hash': '5048d93858a3a7d976ace35f83868215ce5dfe2c'
    },{
        'version': '1.9',
        'tag': '1.9',
        'revision_hash': '3a5c8403d892ff1bc245f5ccc6a00bbc860529eb'
    },{
        'version': '1.10',
        'tag': '1.10',
        'revision_hash': '121b7e00017b95d1498c4f511ceb4dd21811257a'
    },{
        'version': '1.11',
        'tag': '1.11',
        'revision_hash': 'ff38bf57378c7cae8617b3c8df692a5ffbe7b83f'
    },{
        'version': '1.12',
        'tag': '1.12',
        'revision_hash': '3623ee7dad07c74722baeeffcec6770015f915cf'
    },{
        'version': '1.13',
        'tag': '1.13',
        'revision_hash': '45438471726fc5005a41cefecbf62d4d26eb15b2'
    },{
        'version': '1.14',
        'tag': '1.14',
        'revision_hash': 'dd7c7702bf51886aa8bd88b24d98619f310fbeda'
    },{
        'version': '1.15',
        'tag': '1.15',
        'revision_hash': '01b06d5ef5c5ac3bd651bedcfec7433231cea371'
    },{
        'version': '1.16',
        'tag': '1.16',
        'revision_hash': 'c6e38999260bfa486b3213208b809bd543ce18bf'
    }],
    'commons-configuration': [{
        'version': '1.0',
        'tag': 'CONFIGURATION_1_0',
        'revision_hash': '6c460662af216ac3777b03b9d641451486bc238d',
        'corrected_revision': 'a59c5f1b1ee33ddaa2dd719f6fd1a756549f7aca'
    },{
        'version': '1.1',
        'tag': 'CONFIGURATION_1_1',
        'revision_hash': '3480ec5dd93416d3cd804a9d9463aba93e118af0',
        'corrected_revision': 'f50bb53f44c0ac757e1c47258f7c09bf2101b4c9'
    },{
        'version': '1.2',
        'tag': 'CONFIGURATION_1_2',
        'revision_hash': '4c9cc64655258b6af313a593f981de4c94ccc28e',
        'corrected_revision': 'e51302b158a0337e8728d5b94ea02933065c0ffa'
    },{
        'version': '1.3',
        'tag': 'CONFIGURATION_1_3',
        'revision_hash': '663be0ea1296075b919b41c7e3af4c10054c84aa',
        'corrected_revision': '290009d56e2e5abec4e872726fe2b64cfc2d36fa'
    },{
        'version': '1.4',
        'tag': 'CONFIGURATION_1_4',
        'revision_hash': '2ca32d357e00554c999837c52319e8122fcb26d9',
        'corrected_revision': '9724d33bb06a15cf2fc2152d13f9a457b2b07758'
    },{
        'version': '1.5',
        'tag': 'CONFIGURATION_1_5',
        'revision_hash': '6925324fa47ea463e8e7f7833dd2acc4c57ff170'
    },{
        'version': '1.6',
        'tag': 'CONFIGURATION_1_6',
        'revision_hash': '46a1e1cf462d7f3cb89015287cb5cbe67a1ce22b'
    },{
        'version': '1.7',
        'tag': 'CONFIGURATION_1_7',
        'revision_hash': '3bbf2a42b3db2878651a0968574ec375658f8590'
    },{
        'version': '1.8',
        'tag': 'CONFIGURATION_1_8',
        'revision_hash': 'fa910376536ea5a906d3a49a9d87603b11cf5133'
    },{
        'version': '1.9',
        'tag': 'CONFIGURATION_1_9',
        'revision_hash': '93f43a4a6dcc7352888a4853e2178871c3c18523'
    },{
        'version': '1.10',
        'tag': 'CONFIGURATION_1_10',
        'revision_hash': '3c212681e410097cf38ee376540da585a8938923'
    },{
        'version': '2.0',
        'tag': 'CONFIGURATION_2_0',
        'revision_hash': '25c16206e0452a3b68abf4158c4d8f2ea38305da'
    },{
        'version': '2.1',
        'tag': 'CONFIGURATION_2_1',
        'revision_hash': 'e814fe00bb51256386dd78f3f926aa30d8de6a7c'
    },{
        'version': '2.2',
        'tag': 'CONFIGURATION_2_2',
        'revision_hash': '137997974e0ebd3acd047243f46ddf3b90fa0160'
    }],
    'commons-dbcp': [{
        'version': '1.0',
        'tag': 'DBCP_1_0',
        'revision_hash': '68948e66e5b230a256cfb20d1775f7d0d02ac888',
        'corrected_revision': 'ce2d4f6f15b2009f42ec9f66c356edaebc7ad3a2'
    },{
        'version': '1.1',
        'tag': 'DBCP_1_1',
        'revision_hash': '5d76261591d40987acdad7f35b8397e2f63c7916',
        'corrected_revision': '3f7aedad5c09ae61a47c397b50055d935482addf'
    },{
        'version': '1.2',
        'tag': 'DBCP_1_2',
        'revision_hash': '22e7e052373ef9f687064aa307b26aa6ffd46c48',
        'corrected_revision': 'f3bc5838c5e6f173f7665edf00ab7a60b430fa21'
    },{
        'version': '1.3',
        'tag': 'DBCP_1_3',
        'revision_hash': '8162457b6f482d17b73e92710e2fa19b536f1c13',
        'corrected_revision': '306023633625bb8b0d3150ab1d82979f91dca4f9'
    },{
        'version': '1.4',
        'tag': 'DBCP_1_4',
        'revision_hash': 'c08260fd38cc220a8e9c70fd105902f2b5b043b9',
        'corrected_revision': 'ada08866e9390138eb40ebb1a36f4e7fd9bacff9'
    },{
        'version': '2.0',
        'tag': 'DBCP_2_0',
        'revision_hash': '583aaf88451485e75d34c7417a96318e27720e74'
    },{
        'version': '2.1',
        'tag': 'DBCP_2_1',
        'revision_hash': 'd8aa6baa6f1c2ab5281f7097fd11d0d840c4c217'
    },{
        'version': '2.2.0',
        'tag': 'DBCP_2_2_0',
        'revision_hash': 'c5b86d834a53b4891487e62671873d054a6f38d8'
    },{
        'version': '2.3.0',
        'tag': 'DBCP_2_3_0',
        'revision_hash': '6040b67b78d96d05c282a75bb250fa6ea31623dc'
    },{
        'version': '2.4.0',
        'tag': 'commons-dbcp-2.4.0',
        'revision_hash': 'd7aa662fbbb99e536ae28c47d0c4e1d51e39d5b9'
    },{
        'version': '2.5.0',
        'tag': 'commons-dbcp-2.5.0',
        'revision_hash': '334784e3043cadaeaa626f792c3f62ac7b697927'
    }],
    'commons-digester': [{
        'version': '1.0',
        'tag': 'DIGESTER_1_0',
        'revision_hash': '522f50e75b344ebd22343d9b5442f70249e8cabf',
        'corrected_revision': '79fecb6e7fcdac7f854332c07f7898e7a3f56d62'
    },{
        'version': '1.1',
        'tag': 'DIGESTER_1_1',
        'revision_hash': '48bf1c84c58429c545f00446a31d938f1f6b3f48',
        'corrected_revision': '2b782a5a4a11ad4c3ddccf1018a8f2185877c514'
    },{
        'version': '1.2',
        'tag': 'DIGESTER_1_2',
        'revision_hash': 'e70e9c320edcb261f535493f358aa04e8c46c769',
        'corrected_revision': '21255e428189e5c54205d1623077fae3916db9c7'
    },{
        'version': '1.3',
        'tag': 'DIGESTER_1_3',
        'revision_hash': '7533ab004572c33ed076c1137839779c6a00d9ad',
        'corrected_revision': 'db04a200c7cfdd181907c9d4647ecc4536fb3ff7'
    },{
        'version': '1.4',
        'tag': 'DIGESTER_1_4',
        'revision_hash': 'b8f283f82988e616047fcb0b64700f14f7eae3d7',
        'corrected_revision': '4fb91de8c37facd77af6cb80aca4c0e6b99e7884'
    },{
        'version': '1.5',
        'tag': 'DIGESTER_1_5',
        'revision_hash': '4b778ef1af3ebb8ced1687fd904d2bcf33fcca7e',
        'corrected_revision': '05702b56930b9844daa3dea4c2ce56c8341a7000'
    },{
        'version': '1.6',
        'tag': 'DIGESTER_1_6',
        'revision_hash': '619c9da5b28984d736b5d1d12828d17e5c164cab',
        'corrected_revision': 'a9599becb81fb374b48584794cb5890adcf4dc18'
    },{
        'version': '1.7',
        'tag': 'DIGESTER_1_7',
        'revision_hash': '6b775f843092d51f84dc33233ff72d9220ebe3d5',
        'corrected_revision': 'd6e524a56af373dabea5d048d756086a0bf8f255'
    },{
        'version': '1.8',
        'tag': 'DIGESTER_1_8',
        'revision_hash': '8ff0834cbed4fb97f9cf1426993b50342983adeb',
        'corrected_revision': '85ada11c674ef4cb13bf39c10c44a41845055407'
    },{
        'version': '2.0',
        'tag': 'DIGESTER_2_0',
        'revision_hash': '9fb541b5c5f9daa2fa6677914727cc3953515488'
    },{
        'version': '2.1',
        'tag': 'DIGESTER_2_1',
        'revision_hash': 'e2a59fe9a72cf9a2119157cd6859b09230835ba9'
    },{
        'version': '3.0',
        'tag': 'DIGESTER3_3_0',
        'revision_hash': 'ea9f1e1e3bc9c934e70557e8423b2b2c77b4bbfe'
    },{
        'version': '3.1',
        'tag': 'DIGESTER3_3_1',
        'revision_hash': 'fad107b8d66556f915e75b4d0f704dad409b054a'
    },{
        'version': '3.2',
        'tag': 'DIGESTER3_3_2',
        'revision_hash': 'ec75748096c8639be7b4998d99c37bb2e3a9c206'
    }],
    'commons-io': [{
        'version': '1.0',
        'tag': 'IO_1_0',
        'revision_hash': 'c2970b4a557e45a1f6e3cf85b5d163a5de9f80dd',
        'corrected_revision': '5e49391360f799bb42274ce42266c17a484b8f6f'
    },{
        'version': '1.1',
        'tag': 'IO_1_1',
        'revision_hash': '8d911c48d70c93190c8007d915162040a05d89fe',
        'corrected_revision': '1069e5bf6ec3e40f7a9440613615c422da3eb9ac'
    },{
        'version': '1.2',
        'tag': 'IO_1_2',
        'revision_hash': 'd65ff7fec954c84deb0c3b22731b0ee8f395805e',
        'corrected_revision': '1d107c54c400bc76540dd5fcb14207ae56cbf9ad'
    },{
        'version': '1.3',
        'tag': 'IO_1_3',
        'revision_hash': '5a6f8945b08bcb6a4c47cc148a8cbb2f42271880',
        'corrected_revision': 'fb9954d8812defef2ade48bff7003677167e0626'
    },{
        'version': '1.4',
        'tag': 'commons-io-1.4',
        'revision_hash': '6923a01ae1edf0510dd257f72867c282224ee23f'
    },{
        'version': '2.0',
        'tag': 'commons-io-2.0',
        'revision_hash': '27add961ae6f94fa20124d24486a6e5fb47587c9'
    },{
        'version': '2.1',
        'tag': 'commons-io-2.1',
        'revision_hash': '0e68a33a9b8f27e592f79a563a5c9a9644c8b21d'
    },{
        'version': '2.2',
        'tag': '2.2',
        'revision_hash': '5d4d09928e154d083f396a1555da72dfe9d877d1'
    },{
        'version': '2.3',
        'tag': '2.3',
        'revision_hash': 'c07589baa429c2ab306466643cef3dec31d1788b'
    },{
        'version': '2.4',
        'tag': '2.4',
        'revision_hash': 'e36d53170875d26d59ca94bd376bf40bc5690ee6'
    },{
        'version': '2.5',
        'tag': 'commons-io-2.5',
        'revision_hash': '4077158829de92987367d3149e4ba71356bb5390'
    }],
    'commons-jcs': [{
        'version': '1.0',
        'tag': 'JCS_1_0',
        'revision_hash': 'e2349a328c0d17ac6add6e28ea56a16cb640f27a',
        'corrected_revision': '574d315df38c45aefd638c5d22a9d7442635a7d6'
    },{
        'version': '1.1',
        'tag': 'jcs_1_1_dev',
        'revision_hash': '219dece048757b3f04e8d972f40c2c1273b3c54d',
        'corrected_revision': 'f9d4511f36c00fdbe8bde20f5fb6a2805ddf985a'
    },{
        'version': '1.3',
        'tag': 'JCS_1_3',
        'revision_hash': '851ec7399ea430c9be9ec4c8914d839efdd92b10',
        'corrected_revision': '2b4f84253690f2e23337a09be520a76248252418'
    },{
        'version': '2.0',
        'tag': 'commons-jcs-2.0',
        'revision_hash': '8a3cab728832eb739a9697d3a5d2e0184d511511'
    },{
        'version': '2.1',
        'tag': 'commons-jcs-2.1',
        'revision_hash': 'efe68fda0c1c4ef57171e76ef10ca118b96e191a'
    },{
        'version': '2.2',
        'tag': 'commons-jcs-2.2',
        'revision_hash': '5a9246493fb9c60eb81ba4eab7636172e438d46c'
    }],
    'commons-jexl': [{
        'version': '1.0',
        'tag': 'COMMONS_JEXL-1_0',
        'revision_hash': 'd4e5fb3c1e1d09641e7d7fc5418cde5e863a56e8',
        'corrected_revision': 'f3ec15c50f25fcbbb6248c18217d9fe76d1a5f61'
    },{
        'version': '1.1',
        'tag': 'COMMONS_JEXL-1_1',
        'revision_hash': 'b9feb29d0d3abbe0d441685f04014036b71e9106',
        'corrected_revision': '6160d320c4219fd25b2ffd7bb136a1c4a1c53e6c'
    },{
        'version': '2.0',
        'tag': 'COMMONS_JEXL_2_0',
        'revision_hash': '1abdc102c66da73706b2aef331cb57e08f595c04',
        'corrected_revision': 'b9dc71c16461ce497e7ba4b2439983c4d756f0af'
    },{
        'version': '2.1',
        'tag': 'commons-jexl-2.1',
        'revision_hash': '9c630b1b469c6d3badb43c12846d341362994eca'
    },{
        'version': '3.0',
        'tag': 'COMMONS_JEXL_3_0',
        'revision_hash': 'de6c4f3b00af4430f535fcb7833c480d9093fd35'
    },{
        'version': '3.1',
        'tag': 'COMMONS_JEXL_3_1',
        'revision_hash': 'af298b37d30f8cd1e776b23db9bb85c353b002ee'
    }],
    'commons-lang': [{
        'version': '1.0',
        'tag': 'LANG_1_0',
        'revision_hash': '2d274d7cedc148ee3fa8cf4f48904dfd82172bda',
        'corrected_revision': '5091f3b04fbacadc0bcff6aaa808615a2c1f05a3'
    },{
        'version': '2.0',
        'tag': 'LANG_2_0',
        'revision_hash': 'da5d64110a8b7182d04359958f40fadfbe1ed6ff',
        'corrected_revision': '70e09f61dff742883b6b7c6aecb7023cfdabe0bf'
    },{
        'version': '2.1',
        'tag': 'LANG_2_1',
        'revision_hash': '81f1077a922523a1f6a76376215a1b3de1e49b67',
        'corrected_revision': '27d3960eb1d30bc7e4faf03c26b65f788142cc90'
    },{
        'version': '2.2',
        'tag': 'LANG_2_2',
        'revision_hash': 'bdf2ede50ed073d91389be55ec3a9cff565c58a5',
        'corrected_revision': 'f4637c05715359d24979e7eedb3c18dd4b31b599'
    },{
        'version': '2.3',
        'tag': 'LANG_2_3',
        'revision_hash': '4e55808486a0227a72c70350a76d33660cc94af8',
        'corrected_revision': '1363d79e631183ad6c8be6dbe95e895236173d85'
    },{
        'version': '2.4',
        'tag': 'LANG_2_4',
        'revision_hash': 'aa9727e15966bac899aa18733ef85c4aded19f0c'
    },{
        'version': '2.5',
        'tag': 'LANG_2_5',
        'revision_hash': '7c4386cb877d99d733b531f4fba2a328a3951767'
    },{
        'version': '2.6',
        'tag': 'LANG_2_6',
        'revision_hash': '0fe6a434f20b45c027badd89bd393dd4085cf1f2'
    },{
        'version': '3.0',
        'tag': 'LANG_3_0',
        'revision_hash': 'c21484b730221bc87ca26553155350292aa30f0d'
    },{
        'version': '3.1',
        'tag': 'LANG_3_1',
        'revision_hash': '915946c9dcc80a6a16745b86540b82b2ddba1720'
    },{
        'version': '3.2',
        'tag': 'LANG_3_2',
        'revision_hash': '0c14a39be2b8a8e9461921994033310fd567e8cb'
    },{
        'version': '3.3',
        'tag': 'LANG_3_3',
        'revision_hash': '37f8fe84d934a92fe784c62e706b7aacf6fa4f97'
    },{
        'version': '3.4',
        'tag': 'LANG_3_4',
        'revision_hash': '4777c3a5e4291af2420db57d008152c70c4a8f24'
    },{
        'version': '3.5',
        'tag': 'LANG_3_5',
        'revision_hash': '36f98d87b24c2f542b02abbf6ec1ee742f1b158b'
    },{
        'version': '3.6',
        'tag': 'LANG_3_6',
        'revision_hash': '09043bfa6f1f9ebb946c7b159735c83259e3a89f'
    },{
        'version': '3.7',
        'tag': 'LANG_3_7',
        'revision_hash': '425d8085cfcaab5a78bf0632f9ae77b7e9127cf8'
    }],
    'commons-math': [{
        'version': '1.0',
        'tag': 'MATH_1_0',
        'revision_hash': '96691abe5ed4a0f67b253cd2f6f789e868a77b22',
        'corrected_revision': '746ced5ff5d23885fbaff6664d33616941328488'
    },{
        'version': '1.1',
        'tag': 'MATH_1_1',
        'revision_hash': 'bb50284d69ce651dc12e7acea2fe6e2db792874a',
        'corrected_revision': '2e4407d19cef6d91aac716d5d08f5140b13590f7'
    },{
        'version': '1.2',
        'tag': 'MATH_1_2',
        'revision_hash': '0c602508d1591e212c55912a9066baa45e126957'
    },{
        'version': '2.0',
        'tag': 'MATH_2_0',
        'revision_hash': '9f26e58bf9a73e1375b17dfdc2c5f395e4ffbdb6'
    },{
        'version': '2.1',
        'tag': 'MATH_2_1',
        'revision_hash': '7d588288e35b38dd55e7bedb767d1b0d80096294'
    },{
        'version': '2.2',
        'tag': 'MATH_2_2',
        'revision_hash': '98ebcc6bb298e62e35f895d59fca51b8c4c2d443'
    },{
        'version': '3.0',
        'tag': 'MATH_3_0',
        'revision_hash': '174f17d5d4b8251c4ed2600947e69b736ace5877'
    },{
        'version': '3.1',
        'tag': 'MATH_3_1',
        'revision_hash': 'b090abb2cf03cf4d0afc6fbadc7cff62c35077cd'
    },{
        'version': '3.2',
        'tag': 'MATH_3_2',
        'revision_hash': '7ec508828f62c2a3df31cde2a2f3aa32c5135fdb'
    },{
        'version': '3.3',
        'tag': 'MATH_3_3',
        'revision_hash': '38a48e20867f4cd91b7540be4bbba1e9dc7a8b02'
    },{
        'version': '3.4',
        'tag': 'MATH_3_4',
        'revision_hash': 'befd8ebd96b8ef5a06b59dccb22bd55064e31c34'
    },{
        'version': '3.5',
        'tag': 'MATH_3_5',
        'revision_hash': 'b3c5dae8f253fcb4484e5cd3cc5662587803efc2'
    },{
        'version': '3.6',
        'tag': 'MATH_3_6',
        'revision_hash': '95a9d35e77f70ffc9bd5143880c236a760b42005'
    }],
    'commons-net': [{
        'version': '1.0.0',
        'tag': 'NET_1_0_0',
        'revision_hash': 'edb0c4426ab7dc930c29a5eafd3f7ac76c4cd9bf',
        'corrected_revision': '581fad6c1f9bddfe6a61e4fdb2dbc0b6a2709351'
    },{
        'version': '1.1.0',
        'tag': 'NET_1_1_0',
        'revision_hash': 'ce903621374d97ab641ca6e0f07d236236a50970',
        'corrected_revision': 'ace3e9bdc698e2c9c95e7c9103fabb32904896b2'
    },{
        'version': '1.2.0',
        'tag': 'NET_1_2_0',
        'revision_hash': '505a39a18df1507ad63af7a636eaa25b3b793721',
        'corrected_revision': 'c87a7c22668257b4d6073197f53dbd4f08eff631'
    },{
        'version': '1.3.0',
        'tag': 'NET_1_3_0',
        'revision_hash': '4f8f123cda3bebdf22095e80826cf4af5781ceb9',
        'corrected_revision': 'f1481c0e35daf8e45d73698a6f6f4c6651e8dddc'
    },{
        'version': '1.4.0',
        'tag': 'NET_1_4_0',
        'revision_hash': '380f6f7a3bca6819b88656daa4c37f770958d2ac',
        'corrected_revision': '9189ef3ccc6a8c3d6a09c2c25379f98fd315aa7f'
    },{
        'version': '2.0',
        'tag': 'NET_2_0',
        'revision_hash': '32c972d4a86302c8df0be318d9380e8989ea6972'
    },{
        'version': '2.1',
        'tag': 'commons-net-2.1',
        'revision_hash': '5b81ff5f9a6ed63b0e0d67bea7a1d9c909184bdd'
    },{
        'version': '2.2',
        'tag': 'NET_2_2',
        'revision_hash': '51829d33e280cd335b0d48db9e290a82ae574496'
    },{
        'version': '3.0',
        'tag': 'NET_3_0',
        'revision_hash': '7b6def4cb8da5ddef57efdce7f415f7aee1b49e7'
    },{
        'version': '3.1',
        'tag': 'NET_3_1',
        'revision_hash': 'e2fa6168cdbe958e56d54456e7aa1ecc4b13e4df'
    },{
        'version': '3.2',
        'tag': 'NET_3_2',
        'revision_hash': '9f139947e62438f408eb45fcbf8543eaf0d2b922'
    },{
        'version': '3.3',
        'tag': 'NET_3_3',
        'revision_hash': 'f8c2dabb0e8cb62c86912dc61fe2f68106d8b917'
    },{
        'version': '3.4',
        'tag': 'NET_3_4',
        'revision_hash': '74a2282b7e4c6905581f4f1b5a2ec412310cd5e7'
    },{
        'version': '3.5',
        'tag': 'NET_3_5',
        'revision_hash': '943a587727079179b1c42af26b788f1cb5d76643'
    },{
        'version': '3.6',
        'tag': 'NET_3_6',
        'revision_hash': '163fe46c019f5184016207c247cdff30ee740ecc'
    }],
    'commons-scxml': [{
        'version': '0.5',
        'tag': 'SCXML_0_5',
        'revision_hash': 'c5425058630297b6fa8873586b601389e2b80c4b',
        'corrected_revision': 'dcd981be25857db62bd18f43a640c51dedbae65c'
    },{
        'version': '0.6',
        'tag': 'SCXML_0_6',
        'revision_hash': '9407377dad722a129ff7688cce553e103a434212',
        'corrected_revision': '47c4d6f18410985c82bf343fdc50803aa0258f0e'
    },{
        'version': '0.7',
        'tag': 'SCXML_0_7',
        'revision_hash': '439d8722769e199e0475a2e400178594f8b980dc'
    },{
        'version': '0.8',
        'tag': 'SCXML_0_8',
        'revision_hash': '54a019f81af349cb5279c2109bb69f17bdd4de3e'
    },{
        'version': '0.9',
        'tag': 'SCXML_0_9',
        'revision_hash': '836783c31f5546982558584e6395ae26a882534b'
    }],
    'commons-validator': [{
        'version': '1.0',
        'tag': 'VALIDATOR_1_0',
        'revision_hash': '28de1292f039a972cd0fd465e157be34d55d0ac6',
        'corrected_revision': '2909c6197eb48ebe389812dd1d26b59174297e1f'
    },{
        'version': '1.1.0',
        'tag': 'VALIDATOR_1_1_0',
        'revision_hash': '82fb3df34d2c01ff6e6482fd9f84b1363aab6e76',
        'corrected_revision': '11e0d011760e4d72fe5a089e73144efc05eba2ef'
    },{
        'version': '1.2.0',
        'tag': 'VALIDATOR_1_2_0',
        'revision_hash': 'b1833e64d1a5d68ea49db446bf2cce5f4cbc1549',
        'corrected_revision': '1a748629137600611781cbcfde47267763f718da'
    },{
        'version': '1.3.0',
        'tag': 'VALIDATOR_1_3_0',
        'revision_hash': 'de4028bee2c0629aef3fefa42e5128d077ce12f3',
        'corrected_revision': 'b748c1060fc5052e12831c4c174a6ab109bcace7'
    },{
        'version': '1.4.0',
        'tag': 'VALIDATOR_1_4_0',
        'revision_hash': '5eea84edbe4e2f3eeaa9d351bbacb148f5b09373'
    },{
        'version': '1.5.0',
        'tag': 'VALIDATOR_1_5_0',
        'revision_hash': 'ff5b4b45dbea1522510e50dbab2aae65cf47c6d1'
    },{
        'version': '1.6.0',
        'tag': 'VALIDATOR_1_6',
        'revision_hash': 'c4b93a7275b1172f69e53cd8cec7fc27c954a5be'
    }],
    'commons-vfs': [{
        'version': '1.0',
        'tag': 'vfs-1.0',
        'revision_hash': 'ff7dd9697fac9856c8acae2ab06031f83f70e33c',
        'corrected_revision': '8e23674676e3ab8df88513d0051fba6457ccc0c9'
    },{
        'version': '2.0',
        'tag': 'commons-vfs2-project-2.0',
        'revision_hash': '9508889ae6ae432704df3b923d12a6162ba6c89d'
    },{
        'version': '2.1',
        'tag': 'commons-vfs2-project-2.1',
        'revision_hash': 'f159bd4781b5ea91ae8872c2cd729d12511703e9'
    },{
        'version': '2.2',
        'tag': 'commons-vfs2-project-2.2',
        'revision_hash': 'ef1e6d59e29f3658b6d84b540644f4d4b27d0c67'
    }],
    'deltaspike': [{
        'version': '0.1',
        'tag': 'deltaspike-project-0.1-incubating',
        'revision_hash': '6048874eecc016e8d8073fad086382ab2f08312c'
    },{
        'version': '0.2',
        'tag': 'deltaspike-project-0.2-incubating',
        'revision_hash': '5e4a7eb4de01004206f24ae22b9850e643bffe54'
    },{
        'version': '0.3',
        'tag': 'deltaspike-project-0.3-incubating',
        'revision_hash': 'b880838313302a396543f432a91ee005d71a70da'
    },{
        'version': '0.4',
        'tag': 'deltaspike-project-0.4',
        'revision_hash': '3d8af1b1e6726cca4ecd7d087ec79eb1ff943aff'
    },{
        'version': '0.5',
        'tag': 'deltaspike-project-0.5',
        'revision_hash': 'dffe4faa12ef455d5eec4ee29cdec839e041914e'
    },{
        'version': '0.6',
        'tag': 'deltaspike-project-0.6',
        'revision_hash': 'f6a3ad36b974661ca7802986527fdbd95fe113c5'
    },{
        'version': '0.7',
        'tag': 'deltaspike-project-0.7',
        'revision_hash': 'c62a2df8210fa0c18c5fe5a1b907a439c11ae959'
    },{
        'version': '1.0.0',
        'tag': 'deltaspike-project-1.0.0',
        'revision_hash': '015dbba89afeb9bc763da72302587695d41af7e7'
    },{
        'version': '1.1.0',
        'tag': 'deltaspike-project-1.1.0',
        'revision_hash': '6dbd9404fd0c19a030b374322734a78954b5fa93'
    },{
        'version': '1.2.0',
        'tag': 'deltaspike-project-1.2.0',
        'revision_hash': '330cbd1c25824585caad87dc717e74f85c1dcc7b'
    },{
        'version': '1.3.0',
        'tag': 'deltaspike-root-1.3.0',
        'revision_hash': '31c714237829f7e7f07f9d1dc43776f984c78b27'
    },{
        'version': '1.4.0',
        'tag': 'deltaspike-1.4.0',
        'revision_hash': 'af1ed18cfb27e471342c901a496d6ec477a1427d'
    },{
        'version': '1.5.0',
        'tag': 'deltaspike-1.5.0',
        'revision_hash': 'd4caf739411b7ce89eb4ef8f1385c0c346498eb4'
    },{
        'version': '1.6.0',
        'tag': 'deltaspike-1.6.0',
        'revision_hash': '2d55237aebfeda8287d53fa5b6d3e370846f8ab3'
    },{
        'version': '1.7.0',
        'tag': 'deltaspike-1.7.0',
        'revision_hash': '5658d4959c58535b04615ac5ac66bb95a5246e93'
    },{
        'version': '1.8.0',
        'tag': 'deltaspike-1.8.0',
        'revision_hash': '2d4e4099852ffbd6aec1aca73dda331cf5329d91'
    }],
    'eagle': [{
        'version': '0.3.0',
        'tag': 'eagle-0.3.0-incubating',
        'revision_hash': '8183edad341bdec80a7d2acd56a5b5e24f28681c'
    },{
        'version': '0.4.0',
        'tag': 'v0.4.0-incubating',
        'revision_hash': 'eac0f27958f2ed8c6842938dad0a995a87fd0715'
    },{
        'version': '0.5.0',
        'tag': 'v0.5.0',
        'revision_hash': '0b9c91b8788e166ff203e342baf1940d4bd3247c'
    }],
    'giraph': [{
        'version': '0.1.0',
        'tag': 'release-0.1.0',
        'revision_hash': '2adab50db10fbffd94e70dfa8fdb1045c3b9ac8b',
        'corrected_revision': '65c1a337c1de1262ade8b02dc7e868c30f157589'
    },{
        'version': '1.0.0',
        'tag': 'release-1.0.0-RC3',
        'revision_hash': '2537201724b54de25efe144996af0cd762ac6018'
    },{
        'version': '1.1.0',
        'tag': 'release-1.1.0',
        'revision_hash': '0466bccf96ef4c92df4c20a6a4bad62cdf7624c1'
    }],
    'gora': [{
        'version': '0.1',
        'tag': '0.1-incubating',
        'revision_hash': 'a49c0b80c851cb0208c5dc03ab7420e995aec67d',
        'corrected_revision': 'faabec642fa692fadecabc6f345fbd1404e609f8'
    },{
        'version': '0.2',
        'tag': 'gora-0.2',
        'revision_hash': '424a10884ceacf25878b72e9804c6cc6c2ed44e2'
    },{
        'version': '0.3',
        'tag': 'apache-gora-0.3',
        'revision_hash': 'dd6c7a4a20533fba8a9a8aac9ed1e90e0b0633cf'
    },{
        'version': '0.4',
        'tag': 'apache-gora-0.4',
        'revision_hash': '685f26c7db062b28aefae9e2d328fecece295cba'
    },{
        'version': '0.5',
        'tag': 'apache-gora-0.5',
        'revision_hash': 'c2d58dd1440b4e2c66c1f40a4b6d4169d79bb6d3'
    },{
        'version': '0.6',
        'tag': 'apache-gora-0.6',
        'revision_hash': '68ce474b6813e9e5e8b021e3d255f90ddd7a3eb3'
    },{
        'version': '0.7',
        'tag': 'apache-gora-0.7',
        'revision_hash': '3b48688f51d6cf618c760b24a341874f12e160d2'
    },{
        'version': '0.8',
        'tag': 'apache-gora-0.8',
        'revision_hash': 'd173382084f73f4b2779fcb9131498f11f010371'
    }],
    'jspwiki': [{
        'version': '1.4.0',
        'tag': 'jspwiki_1_4_0',
        'revision_hash': '22c1139ea1c48950f54cff5db8ad4636db91d59a',
        'corrected_revision': '52d64cff1939e8f080b275a5504dcddf4c601c82'
    },{
        'version': '1.5.0',
        'tag': 'jspwiki_1_5_0',
        'revision_hash': 'b0eabb7ea312c776a727ef6c63245b2b5d64405f',
        'corrected_revision': '15ef5645e96d38779d850dce8150016f8bd45e83'
    },{
        'version': '1.6.0',
        'tag': 'jspwiki_1_6_0',
        'revision_hash': 'f9eae56bb2002461fa2499a445845bf132b462fb',
        'corrected_revision': '63caf8f3f30ab8432f2807fa459a66c6038759c8'
    },{
        'version': '1.7.0',
        'tag': 'jspwiki_1_7_0',
        'revision_hash': 'c784eef937df2cf318e1a8c06426fac324b60303',
        'corrected_revision': '196dab3ce4fb0ef3a58d6a35aeab7f51219a7577'
    },{
        'version': '1.8.0',
        'tag': 'jspwiki_1_8_0',
        'revision_hash': '96b60a83f3b0b03aa4f5da11ea70261d0af54e6b',
        'corrected_revision': 'cdc5345ff16e70350ac30fe43b1d0da4a71b3a14'
    },{
        'version': '2.0.36',
        'tag': 'jspwiki_2_0_36',
        'revision_hash': '5c46f2772e69315b5aae215dbaf3bea73b17f9eb',
        'corrected_revision': '14d5a0542f4c22d03e697b233c02aaa2f7c3891b'
    },{
        'version': '2.2.19',
        'tag': 'jspwiki_2_2_19',
        'revision_hash': '99a009637d47c0f57bd50932a65e8d92a572dcc2',
        'corrected_revision': '3af8a6b81158db4a1d39cfa7965e4dd08156635a'
    },{
        'version': '2.4.56',
        'tag': 'jspwiki_2_4_56',
        'revision_hash': '6bc63555b9eaf04878e2ce832b8bdf57fa820358',
        'corrected_revision': '7da437345960f35936dacddd4398d49628b41067'
    },{
        'version': '2.6.0',
        'tag': 'jspwiki_2_6_0',
        'revision_hash': '7f8c385f794a404b24bd412f2823e811948086fa',
        'corrected_revision': 'd2e78bf3403cadda7b66a1caa71ff9c949c8a580'
    },{
        'version': '2.8.0',
        'tag': 'jspwiki_2_8_0',
        'revision_hash': '3e85477ac133e31b85c5713cbcc6c3aaaf4d814e',
        'corrected_revision': 'e3246df9966adc64bf507206f90e3b324e47d982'
    },{
        'version': '2.9.0',
        'tag': 'jspwiki_2_9_0_incubating',
        'revision_hash': '7eaed006f8585a3be13508ccc66e316687194f8d',
        'corrected_revision': 'b52ef69fd517494a3f7856d8cdffd34d5247ba39'
    },{
        'version': '2.10.0',
        'tag': 'jspwiki_2_10_0',
        'revision_hash': 'ec0350a26858f25702c59dfaf5828142428b7929'
    }],
    'knox': [{
        'version': '0.3.0',
        'tag': 'v0.3.0-final',
        'revision_hash': 'd806123b3fda3f7149ad0b69b921f078a8360b37'
    },{
        'version': '0.4.0',
        'tag': 'v0.4.0-release',
        'revision_hash': '614c54d9b236bac81e3685b475264115d5e26b77'
    },{
        'version': '0.5.0',
        'tag': 'v05.0-release',
        'revision_hash': 'f29d530df008901e86317a9135f3e57de74166b3'
    },{
        'version': '0.6.0',
        'tag': 'v0.6.0-release',
        'revision_hash': 'fd88b115502f4b9fc9f3efae4b3c54d6e64246aa'
    },{
        'version': '0.7.0',
        'tag': 'v0.7.0-release',
        'revision_hash': '4f86a8e6b5a581ff46f6188693dd3cf5f4755e7b'
    },{
        'version': '0.8.0',
        'tag': 'v0.8.0-release',
        'revision_hash': '1473284f692ea14360bf790b4859d207df060e02'
    },{
        'version': '0.9.0',
        'tag': 'v0.9.0-release',
        'revision_hash': 'a1db6ee4028274f7959c626b73934cb79bb2493a'
    },{
        'version': '0.10.0',
        'tag': 'v0.10.0-release',
        'revision_hash': '905b1f247ce9682ae2af92468c514d41fcd8d059'
    },{
        'version': '0.11.0',
        'tag': 'v0.11.0-release',
        'revision_hash': '074c110cd84995b55141ece46fa44c689968a041'
    },{
        'version': '0.12.0',
        'tag': 'v0.12.0-release',
        'revision_hash': 'ee4b1473aa7570092f80d84b224f773111602835'
    },{
        'version': '0.13.0',
        'tag': 'v0.13.0-release',
        'revision_hash': '15f73ca81382b3c9edf31245bd525d9efdaf8997'
    },{
        'version': '0.14.0',
        'tag': 'v0.14.0-release',
        'revision_hash': 'e32ca83569a4d811cc7c16616e5cda837078fd08'
    },{
        'version': '1.0.0',
        'tag': 'v1.0.0-release',
        'revision_hash': '3c3f23a866d55de563c6574353019858590367d8'
    }],
    'kylin': [{
        'version': '0.6.1',
        'tag': 'v0.6.1',
        'revision_hash': '72138e235a307ac71061421d31b802969263e4ab'
    },{
        'version': '0.7.1',
        'tag': 'kylin-0.7.1-incubating',
        'revision_hash': 'a6e24b91717bc2e4e01873e1ce062c67a7710115'
    },{
        'version': '1.0',
        'tag': 'kylin-1.0-incubating',
        'revision_hash': 'f8a3320a85f6e26d85f3bffed73c0d38313c611c'
    },{
        'version': '1.1',
        'tag': 'kylin-1.1-incubating',
        'revision_hash': 'e8f226f2e8127dabef79a36420dc8724d0852688'
    },{
        'version': '1.2',
        'tag': 'kylin-1.2',
        'revision_hash': '0fc9be513ccc1d62f647b70d1b310d4013d1e0b0'
    },{
        'version': '1.3',
        'tag': 'kylin-1.3',
        'revision_hash': '75355046859dbb63df3e176fc63759f829a19769'
    },{
        'version': '1.5.0',
        'tag': 'kylin-1.5.0',
        'revision_hash': 'e1e732c0aa233123dc6a9d864e72b10aee0fd86e'
    },{
        'version': '1.6.0',
        'tag': 'kylin-1.6.0',
        'revision_hash': 'ed6982c8b3baaad08b7e4956001339979724d9a7'
    },{
        'version': '2.0.0',
        'tag': 'kylin-2.0.0',
        'revision_hash': '375fd807c281d8c5deff0620747c806be2019782'
    },{
        'version': '2.1.0',
        'tag': 'kylin-2.1.0',
        'revision_hash': '562dd173aaf6b398be8e053f896755b3afe8137f'
    },{
        'version': '2.2.0',
        'tag': 'kylin-2.2.0',
        'revision_hash': '850c0a6a3b9296d26121f9986701bb191921b1bf'
    }],
    'lens': [{
        'version': '2.6.0',
        'tag': 'apache-lens-2.6.0',
        'revision_hash': '925094f44923b33767a9d1d4bc9ea6b2bb3fe229'
    },{
        'version': '2.7.0',
        'tag': 'apache-lens-2.7.0',
        'revision_hash': 'd2b49929dd2af6a401a8ea6fa2d357c270a6890e'
    }],
    'mahout': [{
        'version': '0.1',
        'tag': 'mahout-0.1',
        'revision_hash': 'bee952edafdfb517191d3013d8d29a7d574c6a51',
        'corrected_revision': '56840d66ec7e29033395f8440f509d469e1d59a3'
    },{
        'version': '0.2',
        'tag': 'mahout-0.2',
        'revision_hash': '7a786023136ab3c6443f6aca4556a71e093b5ff2',
        'corrected_revision': '205296876dd21ef387d92843b3052c54bdeef972'
    },{
        'version': '0.3',
        'tag': 'mahout-0.3',
        'revision_hash': 'f73d4002aec1844622829f2c2822ea4765ec73b9',
        'corrected_revision': '514cb7c8deb753f9fb36c70f9dc8302a04cb67a8'
    },{
        'version': '0.4',
        'tag': 'mahout-0.4',
        'revision_hash': 'ce337a9c20c48c52c1680a4d1074a943c57c2107'
    },{
        'version': '0.5',
        'tag': 'mahout-0.5',
        'revision_hash': 'e597fcdb0465cb3f23ebc30266bc16993205c5d3'
    },{
        'version': '0.6',
        'tag': 'mahout-0.6',
        'revision_hash': '17a286d0403eed3e8c3c8cb2aaa815d0bce0bbe9'
    },{
        'version': '0.7',
        'tag': 'mahout-0.7',
        'revision_hash': '9d847e0357a2138687808ee257e53455ae4786c3'
    },{
        'version': '0.8',
        'tag': 'mahout-0.8',
        'revision_hash': '86573002086f6f0381c4aa927efac76608cd7f0a'
    },{
        'version': '0.9',
        'tag': 'mahout-0.9',
        'revision_hash': 'a41d378df344330192c88ea9d63e248bd8182f5c'
    },{
        'version': '0.10.0',
        'tag': 'mahout-0.10.0',
        'revision_hash': '832b08c40752b58a98e1a89891a1e0ff54094910'
    },{
        'version': '0.11.0',
        'tag': 'mahout-0.11.0',
        'revision_hash': 'b3a44a4ab274266a0a0808f8c401e80059db48ac'
    },{
        'version': '0.12.0',
        'tag': 'mahout-0.12.0',
        'revision_hash': '75c382f2129544a26d35889df7c15ab9d5ee9104'
    },{
        'version': '0.13.0',
        'tag': 'mahout-0.13.0',
        'revision_hash': 'cca62c5a600703b99a6291440193dd233103d4e4'
    }],
    'manifoldcf': [{
        'version': '0.1',
        'tag': 'release-0.1-incubating',
        'revision_hash': '61f1b3abaaa5a15527957f83d0b6d759704b2f56',
        'corrected_revision': '59eef781b5cdae0d057e03c301ad296e7e2a0310'
    },{
        'version': '0.2',
        'tag': 'release-0.2-incubating',
        'revision_hash': 'd87df29245c3ddbe06b3d53341a9194d6ce72183',
        'corrected_revision': 'eac4057fffd591c7987acb8e440f5f91edf74865'
    },{
        'version': '0.3',
        'tag': 'release-0.3-incubating',
        'revision_hash': 'da5a3251bd43613782296497279d9c6c4fd30018',
        'corrected_revision': 'fd3df0934f7d2f342090cf57edf90b0cf88216f1'
    },{
        'version': '0.4',
        'tag': 'release-0.4-incubating',
        'revision_hash': '29edf34ddfe813f2b7a4a48e76a2d2aa696c232b',
        'corrected_revision': 'efc76d2e36ee6aa12339193972ffbd9524c7db8b'
    },{
        'version': '0.5',
        'tag': 'release-0.5-incubating',
        'revision_hash': '588465f8b9c0cf47c5eb9d42bea8833f80c6bfc9',
        'corrected_revision': 'aa2caf8111305d1a06c93ecfee27b3133e49b266'
    },{
        'version': '0.6',
        'tag': 'release-0.6',
        'revision_hash': 'c8c76f505940e722788aa64522887a797df6c7e1'
    },{
        'version': '1.0',
        'tag': 'release-1.0',
        'revision_hash': '4d9ec8e51e136ca3619463f56948456882b5f6e1'
    },{
        'version': '1.1',
        'tag': 'release-1.1',
        'revision_hash': 'd270aeb0619d179b9c2a79104a07c1693d902163'
    },{
        'version': '1.2',
        'tag': 'release-1.2',
        'revision_hash': 'f1251713c5dd9eefebb4ccad80958bf3ec807458'
    },{
        'version': '1.3',
        'tag': 'release-1.3',
        'revision_hash': '0ab44af1e19de90a4660b115554729fb1f3bc0e9'
    },{
        'version': '1.4',
        'tag': 'release-1.4',
        'revision_hash': '1ef189ad890c01fdf42b7bd9acf5dca3d0a02acf'
    },{
        'version': '1.5',
        'tag': 'release-1.5',
        'revision_hash': 'ca4abe5982c4187098abfd32f6ed6eb4d3f07404'
    },{
        'version': '1.6',
        'tag': 'release-1.6',
        'revision_hash': 'da4f8d10a7b17ead1b68160b14064eced7ad242f'
    },{
        'version': '1.7',
        'tag': 'release-1.7',
        'revision_hash': '9d1bdfd510307f011ce03c48ae11e3527755c59a'
    },{
        'version': '1.8',
        'tag': 'release-1.8',
        'revision_hash': 'ccbafe553777f5c54f7efebf6e1821e18e9c321d'
    },{
        'version': '1.9',
        'tag': 'release-1.9',
        'revision_hash': 'ea504faaf95fcb0a2077b5d761a2520756d4df9c'
    },{
        'version': '1.10',
        'tag': 'release-1.10',
        'revision_hash': '14d42d2668bd4314dc65b1fe3a40f6b8b551eb28'
    },{
        'version': '2.0',
        'tag': 'release-2.0',
        'revision_hash': '9565b0ddcfa09baff2cfbfefe870aad4015df290'
    },{
        'version': '2.1',
        'tag': 'release-2.1',
        'revision_hash': '9cd4fe99ec8eb1ce7818cc0539524e9daac3143d'
    },{
        'version': '2.2',
        'tag': 'release-2.2',
        'revision_hash': '925f56d102ae25ae2e1c88a1d00208cada999dc0'
    },{
        'version': '2.3',
        'tag': 'release-2.3',
        'revision_hash': 'e701989191c61b0551eceb3c7e576cb7e062d2bf'
    },{
        'version': '2.4',
        'tag': 'release-2.4-RC0',
        'revision_hash': 'ca2e2454253e2d7b711f3ac8ce2b171a16e1e9ca'
    },{
        'version': '2.5',
        'tag': 'release-2.5',
        'revision_hash': '25f015a7a1e2b92a10f3f89df0462b9aaf7f1a94'
    },{
        'version': '2.6',
        'tag': 'release-2.6',
        'revision_hash': '46841cb51a195e5a1c12ee94e074c17436941382'
    },{
        'version': '2.7',
        'tag': 'release-2.7',
        'revision_hash': '19396df543b59c7c9dbf9bcc8e8dc8ba6a53351c'
    },{
        'version': '2.8',
        'tag': 'release-2.8',
        'revision_hash': '115d85b056a832c9f6a89156ecaae87fb006be44'
    },{
        'version': '2.9',
        'tag': 'release-2.9',
        'revision_hash': 'b5ce53b8f1fdf6dade99b9b6f7191f6042edeeda'
    },{
        'version': '2.10',
        'tag': 'release-2.10',
        'revision_hash': '46e349207d275410276cbb2540de77aa8ad3aab3'
    }],
    'nutch': [{
        'version': '0.7',
        'tag': 'release-0.7',
        'revision_hash': 'c089c9aa676d373fd6469d277b92b3438d263544',
        'corrected_revision': 'e3ad07e5a6de78d1088f0ed459edf0da5d519a45'
    },{
        'version': '0.8',
        'tag': 'release-0.8',
        'revision_hash': '616425eacb1ba37652e64a180ad8ae4dd2e618c0',
        'corrected_revision': '5ec47e73a47d7d9f963335ec327ceca7ec403fc9'
    },{
        'version': '0.9',
        'tag': 'release-0.9',
        'revision_hash': '1742e5a8eee6e30faad250c29be4f1ae138e741c',
        'corrected_revision': '3988e7cf047fb862264c670979eb52d99c83fbed'
    },{
        'version': '1.0',
        'tag': 'release-1.0',
        'revision_hash': '4f62826994129225e6f8999763677c0e4e29d67e',
        'corrected_revision': '4dd40032ae99fa8c0c36958064a66eae5c2f0606'
    },{
        'version': '1.1',
        'tag': '1.1',
        'revision_hash': 'eae9058af45f3b79dd108e326da2ab915f7b3284',
    },{
        'version': '1.2',
        'tag': 'release-1.2',
        'revision_hash': '5c39b8d4106202f06f7b5782452493f77314ddb8',
    },{
        'version': '1.3',
        'tag': 'release-1.3',
        'revision_hash': '8859cb5d63da7196fa72887e6022ba6d1d69bf82',
    },{
        'version': '1.4',
        'tag': 'release-1.4',
        'revision_hash': 'f4c7f5f6e77fa3a6737c13b5d32278cfe52155d5',
    },{
        'version': '1.5',
        'tag': 'release-1.5',
        'revision_hash': '087b12586cced83f42a8b31c9c596448b9b1f756',
    },{
        'version': '1.6',
        'tag': 'release-1.6',
        'revision_hash': 'd25863a32517fbcb090d2a6944b4e5560369010c',
    },{
        'version': '1.7',
        'tag': 'release-1.7',
        'revision_hash': '0b9c368d86824c20cfb7fe34d1ad5a30210a9abc',
    },{
        'version': '1.8',
        'tag': 'release-1.8',
        'revision_hash': '290a9ae2727bcce0a153657c80f22e2e26287cc2',
    },{
        'version': '1.9',
        'tag': 'release-1.9',
        'revision_hash': '1a865fcd43f15782fe599b941cd2e1785df4bfc7',
    },{
        'version': '1.10',
        'tag': 'release-1.10',
        'revision_hash': 'a03415029f68cfe844ade42d1d3f0e604b13675c',
    },{
        'version': '1.11',
        'tag': 'release-1.11-rc2',
        'revision_hash': '7d0a9855b3c4686382c60df6e39ca01dc3210690',
    },{
        'version': '1.12',
        'tag': 'release-1.12',
        'revision_hash': '7956daee8ac91180070f92949ecf99deae9b5ef0',
    },{
        'version': '1.13',
        'tag': 'release-1.13',
        'revision_hash': 'bd0da3569aa14105799ed39204d4f0a31c77b42c',
    },{
        'version': '1.14',
        'tag': 'release-1.14',
        'revision_hash': 'a8e60bdfb79b368612f068ed5aeeb690e29b448d',
    },{
        'version': '2.0',
        'tag': 'release-2.0',
        'revision_hash': '7d8b5e5b5d930e0f31ce28c725d53a275d971e22',
    },{
        'version': '2.1',
        'tag': 'release-2.1',
        'revision_hash': 'f3d1202d3932c5cb92485c799ef8d591b8abf8cb',
    },{
        'version': '2.2',
        'tag': 'release-2.2',
        'revision_hash': 'b8b1cdf17e41e88c292cf3afc67fdd4e820170f7',
    },{
        'version': '2.3',
        'tag': 'release-2.3',
        'revision_hash': '2578f9f219b3ecbef52873f7f25bd56b4818c343',
    }],
    'opennlp': [{
        'version': '1.7.0',
        'tag': 'opennlp-1.7.0',
        'revision_hash': '2083525415df4f5268abe265688d6ad173b54daf',
    },{
        'version': '1.8.0',
        'tag': 'opennlp-1.8.0',
        'revision_hash': '73c8e5b9d8e055fefb53f7f3c2487d05c9788c6a',
    }],
    'parquet-mr': [{
        'version': '1.0.0',
        'tag': 'parquet-1.0.0',
        'revision_hash': '6eec81d8abab0644e8c2df00544f73828ee0e219',
    },{
        'version': '1.1.0',
        'tag': 'parquet-1.1.0',
        'revision_hash': 'bee8378a90bee9657526cd872c9173309b0bafc2',
    },{
        'version': '1.2.0',
        'tag': 'parquet-1.2.0',
        'revision_hash': '4170539b9a9bca97e4f3159113606855a41c3b0f',
    },{
        'version': '1.3.0',
        'tag': 'parquet-1.3.0',
        'revision_hash': '87864cbd1f0ce52d4e357c02ca522f5309061caa',
    },{
        'version': '1.4.0',
        'tag': 'parquet-1.4.0',
        'revision_hash': '73d66174032ebba5274a7faa172542b4d70b093a',
    },{
        'version': '1.5.0',
        'tag': 'parquet-1.5.0',
        'revision_hash': '79977453b8cd65e6244f16316fac3a510aa87aa8',
    },{
        'version': '1.6.0',
        'tag': 'apache-parquet-mr-1.6.0-incubating',
        'revision_hash': '4f660778f89d4164c72676f6b15bbc74c0d09373',
    },{
        'version': '1.7.0',
        'tag': 'apache-parquet-1.7.0',
        'revision_hash': '32c46643845ea8a705c35d4ec8fc654cc8ff816d',
    },{
        'version': '1.8.0',
        'tag': 'apache-parquet-1.8.0',
        'revision_hash': '0fda28af84b9746396014ad6a415b90592a98b3b',
    },{
        'version': '1.9.0',
        'tag': 'apache-parquet-1.9.0',
        'revision_hash': '38262e2c80015d0935dad20f8e18f2d6f9fbd03c',
    }],
    'santuario-java': [{
        'version': '1.0.0',
        'tag': 'v1_0_0',
        'revision_hash': 'a0ac8137fe49b96edff2011da786ae579891fa75',
        'corrected_revision': '3aad6d8bcdcacb67bf129379f6417bf6199c461f'
    },{
        'version': '1.2',
        'tag': 'v1_2j',
        'revision_hash': 'a0ac8137fe49b96edff2011da786ae579891fa75',
        'corrected_revision': '3aad6d8bcdcacb67bf129379f6417bf6199c461f'
    },{
        'version': '1.4.5',
        'tag': '1.4.5',
        'revision_hash': '8d1b69a2901b241d913e4d2b12028f0ee5351ceb',
    },{
        'version': '1.5.9',
        'tag': '1.5.0',
        'revision_hash': '2d64c5b24af718f184e0a352480d6e85a36bb71a',
    },{
        'version': '2.0.0',
        'tag': 'xmlsec-2.0.0',
        'revision_hash': 'bddfc74628be4412711cc4a930468df7ef373597',
    },{
        'version': '2.1.0',
        'tag': 'xmlsec-2.1.0',
        'revision_hash': '74ebb633b441487707ec27ed60ab33803869c3f9',
    }],
    'systemml': [{
        'version': '0.9',
        'tag': None,
        'revision_hash': None,
        'corrected_revision': '630fa2ea845c5af1d765c7f65477e7e1ab582dc6',
    },{
        'version': '0.10',
        'tag': None,
        'revision_hash': None,
        'corrected_revision': '3d5f9b11741f6d6ecc6af7cbaa1069cde32be838',
    },{
        'version': '0.11',
        'tag': None,
        'revision_hash': None,
        'corrected_revision': 'c2e1670c2745863195d4789f1f77ed01ec11af5e',
    },{
        'version': '0.12',
        'tag': None,
        'revision_hash': None,
        'corrected_revision': 'a83ae1f7fbe9551da674450eedce08aaf7d19d07',
    },{
        'version': '0.13',
        'tag': None,
        'revision_hash': None,
        'corrected_revision': 'ff3e741694e507f64a6b52ee71638bddecabe7af',
    },{
        'version': '0.14',
        'tag': None,
        'revision_hash': None,
        'corrected_revision': '8bdcf106ca9bd04c0f68924ad5827eb7d7d54952',
    },{
        'version': '0.15',
        'tag': None,
        'revision_hash': None,
        'corrected_revision': 'ccaaa18769ae045fd31068dead2565844b4cee47',
    },{
        'version': '1.0.0',
        'tag': None,
        'revision_hash': None,
        'corrected_revision': '6519f3383f8fd43af93ecba8742ae4b2b28d4b35',
    },{
        'version': '1.1.0',
        'tag': None,
        'revision_hash': None,
        'corrected_revision': 'deddaee1fd3f8e87ff3a4403edf06cdb022ba949',
    },{
        'version': '1.2.0',
        'tag': None,
        'revision_hash': None,
        'corrected_revision': 'a1a05e29f6ee78f3c33fea355f62c78ce21766ee',
    }],
    'tika': [{
        'version': '0.1',
        'tag': '0.1-incubating',
        'revision_hash': 'fe7399ebe5f1d2cca553252577d0925815072b21',
        'corrected_revision': '2ca661e4cf4f5a21a53feec0074bf376fa17a93e'
    },{
        'version': '0.2',
        'tag': '0.2',
        'revision_hash': '938e8f87779351e071a6d24106af116d24bac07d',
        'corrected_revision': 'f03c0cae77843869abad37b0e5bf6f0efbbf60f7'
    },{
        'version': '0.3',
        'tag': '0.3',
        'revision_hash': 'b76f7f8836a744bfd1590c046f293c04f1cad2f4',
        'corrected_revision': '9ae0addd0de3a6f9042529705e5410bfbf1e7552'
    },{
        'version': '0.4',
        'tag': '0.4',
        'revision_hash': '02070084a054a7a1e325a2f54a145439c44e2df0',
        'corrected_revision': '163b0a4928cd9c9585406deac353d5f40306bd29'
    },{
        'version': '0.5',
        'tag': '0.5',
        'revision_hash': '450ffaca1c8beba7c1fc6cef047c89e5dfe6c5b0',
        'corrected_revision': 'c05a953cc692ccca7f0909efb8a2aaf19b9b806a'
    },{
        'version': '0.6',
        'tag': '0.6',
        'revision_hash': '9567efeb7087151074abcd2b6fbf6a72a8d2feee',
        'corrected_revision': '4912160b80b9771d1176f5132564d3fd5c71957a'
    },{
        'version': '0.7',
        'tag': '0.7',
        'revision_hash': '69772569d54dd208d0a07585f406baab76874c92',
        'corrected_revision': '6009758401347098a2660505c74b4bb74f2302c4'
    },{
        'version': '0.8',
        'tag': '0.8',
        'revision_hash': '107c4ef06420b27caa099f052aa3852e00ff3aa0'
    },{
        'version': '0.9',
        'tag': '0.9',
        'revision_hash': 'eb1cd6a1665a9034195b30172c0199a6143bcd45'
    },{
        'version': '0.10',
        'tag': '0.10',
        'revision_hash': '2138e990ec9cc7690ad54f574d623f48dcd72ae1'
    },{
        'version': '1.0',
        'tag': '1.0',
        'revision_hash': '6d7139b7f9bc73ee3797bb43145cf87264fc49cd'
    },{
        'version': '1.1',
        'tag': '1.1',
        'revision_hash': '0f8a1292c2f243442030161fecf9fa04aae89727'
    },{
        'version': '1.2',
        'tag': '1.2',
        'revision_hash': 'de47325be3c9a7aff20075fa4354a9be221afffd'
    },{
        'version': '1.3',
        'tag': '1.3',
        'revision_hash': 'f0027f53493d4cada75b48e38faa5d32675b7034'
    },{
        'version': '1.4',
        'tag': '1.4',
        'revision_hash': '540d4166a963ae9f7a0c1a18c7b1cfa21c245e10'
    },{
        'version': '1.5',
        'tag': '1.5',
        'revision_hash': 'a79ed8f00d6329ac9aaa94747923f36af654b129'
    },{
        'version': '1.6',
        'tag': '1.6',
        'revision_hash': '5ee61497debd6c5e731c871c4f75f79cfbd37b14'
    },{
        'version': '1.7',
        'tag': '1.7',
        'revision_hash': 'f9003a1fe1a1883008b62e78d01a582be6c06a8c'
    },{
        'version': '1.8',
        'tag': '1.8',
        'revision_hash': 'c57e805ee506123afc208eae5fd97fb4c4d1b8f7'
    },{
        'version': '1.9',
        'tag': '1.9-rc2',
        'revision_hash': '49757b678139ab8501a1585154cc004bfe080606'
    },{
        'version': '1.10',
        'tag': '1.10',
        'revision_hash': '3701de89d12cb9d6342887521aec5b1f1b141e84'
    },{
        'version': '1.11',
        'tag': '1.11-rc1',
        'revision_hash': '3610d7db1ec6b75ca1927c49b97452333a236a0f'
    },{
        'version': '1.12',
        'tag': '1.12-rc1',
        'revision_hash': 'c0d2b4fc5a8b06d3939253853e0fed005fd7771a'
    },{
        'version': '1.13',
        'tag': '1.13',
        'revision_hash': '386b68b5ae87beafacfb63f33e0a9888dedb9c30'
    },{
        'version': '1.14',
        'tag': '1.14-rc1',
        'revision_hash': 'dbb6baad77937db89d5ac05d4dea0b246fb73a6d'
    },{
        'version': '1.15',
        'tag': '1.15',
        'revision_hash': 'd831efbea89457300dbbd452caeb9790d4f47a65'
    },{
        'version': '1.16',
        'tag': '1.16',
        'revision_hash': 'b99f344f6d462f1ef5cfd0e864703a2d42f8752e'
    },{
        'version': '1.17',
        'tag': '1.17',
        'revision_hash': 'b054df2be2f43c4944ff66f7d9fd2ceb1655d9c0'
    }],
    'wss4j': [{
        'version': '1.5.0',
        'tag': '1_5_0',
        'revision_hash': '1076b70d62584310a22636532fcf4af68fe96554'
    },{
        'version': '1.6.0',
        'tag': '1_6_0',
        'revision_hash': '3db427f29876d4920faf77a613e488113e59e0f2'
    },{
        'version': '2.0.0',
        'tag': 'wss4j-2.0.0',
        'revision_hash': '181d94d615489e3dbe59be38dd465f0b9e8af007'
    },{
        'version': '2.1.0',
        'tag': 'wss4j-2.1.0',
        'revision_hash': '2a2d11a10260fac80995e3da10657de8b8347516'
    },{
        'version': '2.2.0',
        'tag': 'wss4j-2.2.0',
        'revision_hash': '1df91ee6048ffd7d7e3afee78122a6aa22ebe9e9'
    }]
}

In [6]:
all_releases = []
for project in projects:
    for release in releases[project]:
        all_releases.append(project+'-'+release['version'])

project_data = pd.DataFrame(index=projects, columns=['Releases', 'Commits', 'Issues', 'Bugs', 'Linked Bugs', 'Homepage', 'VCS System', 'Issue System'])
release_data = pd.DataFrame(index=all_releases, columns=['Release Date', '#Files Total', '#Files Production'])



for project in projects:
    cur_releases = []
    for release in releases[project]:
        cur_releases.append(release['version'])
        if 'corrected_revision' in release:
            revision_hash = release['corrected_revision']
        else:
            revision_hash = release['revision_hash']
        release_commit = Commit.objects(revision_hash=revision_hash).get()
        files_total = 0
        files_production = 0
        for ces in CodeEntityState.objects(id__in=release_commit.code_entity_states, ce_type='file').only('long_name'):
            files_total += 1
            if java_filename_filter(ces.long_name):
                files_production += 1
        release_data.at[project+'-'+release['version'],'Release Date'] = release_commit.committer_date
        release_data.at[project+'-'+release['version'],'#Files Total'] = files_total
        release_data.at[project+'-'+release['version'],'#Files Production'] = files_production
    releases_str = str(cur_releases)
    releases_str = releases_str.replace('[','')
    releases_str = releases_str.replace(']','')
    releases_str = releases_str.replace("'",'')
    project_id = Project.objects(name=project).get().id
    vcs_system = VCSSystem.objects(project_id=project_id).get()
    its_system = IssueSystem.objects(project_id=project_id).get()
    num_commits = Commit.objects(vcs_system_id=vcs_system.id).count()
    num_issues = Issue.objects(issue_system_id=its_system.id).count()
    num_bugs = 0
    num_linked_bugs = 0
    for issue in Issue.objects(issue_system_id=its_system.id,issue_type__iexact='bug'):
        if jira_is_resolved_and_fixed(issue):
            num_bugs += 1
            if Commit.objects(linked_issue_ids=issue.id).count()>0:
                num_linked_bugs += 1
    
    project_data.at[project,'Releases'] = releases_str
    project_data.at[project,'Commits'] = num_commits
    project_data.at[project,'Issues'] = num_issues
    project_data.at[project,'Bugs'] = num_bugs
    project_data.at[project,'Linked Bugs'] = num_linked_bugs
    project_data.at[project,'Homepage'] = homepages[project]
    project_data.at[project,'VCS System'] = vcs_system.url
    project_data.at[project,'Issue System'] = its_system.url
    
print('Number of Projects:'.ljust(25), len(project_data))
print('Number of Releases:'.ljust(25), len(release_data))
print('Total commits:'.ljust(25), project_data['Commits'].sum())
print('Total issues:'.ljust(25), project_data['Issues'].sum())
print('Total bugs:'.ljust(25), project_data['Bugs'].sum())
print('Total linked bugs:'.ljust(25), project_data['Linked Bugs'].sum())

display(project_data)
display(release_data)

Number of Projects:       38
Number of Releases:       398
Total commits:            140442
Total issues:             45323
Total bugs:               15283
Total linked bugs:        11295


,Releases,Commits,Issues,Bugs,Linked Bugs,Homepage,VCS System,Issue System
ant-ivy,"1.4.1, 2.0.0, 2.1.0, 2.2.0, 2.3.0, 2.4.0",3189,1585,608,535,https://ant.apache.org/ivy/,https://github.com/apache/ant-ivy,https://issues.apache.org/jira/rest/api/2/sear...
archiva,"1.0, 1.1, 1.2, 1.3, 2.0.0, 2.1.0, 2.2.0",10261,2010,751,542,https://archiva.apache.org/,https://github.com/apache/archiva.git,https://issues.apache.org/jira/rest/api/2/sear...
calcite,"1.0.0, 1.1.0, 1.2.0, 1.3.0, 1.4.0, 1.5.0, 1.6....",2926,2608,1178,842,https://calcite.apache.org/,https://github.com/apache/calcite.git,https://issues.apache.org/jira/rest/api/2/sear...
cayenne,"3.0.0, 3.1.0",6619,2430,849,530,https://cayenne.apache.org/,https://github.com/apache/cayenne.git,https://issues.apache.org/jira/rest/api/2/sear...
commons-bcel,"5.0, 5.1, 5.2, 6.0, 6.1, 6.2",1429,316,149,53,https://commons.apache.org/bcel/,https://github.com/apache/commons-bcel.git,https://issues.apache.org/jira/rest/api/2/sear...
commons-beanutils,"1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7.0, 1.8....",1341,523,150,76,https://commons.apache.org/beanutils/,https://github.com/apache/commons-beanutils.git,https://issues.apache.org/jira/rest/api/2/sear...
commons-codec,"1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 1...",1838,258,93,64,https://commons.apache.org/codec/,https://github.com/apache/commons-codec.git,https://issues.apache.org/jira/rest/api/2/sear...
commons-collections,"1.0, 2.0, 2.1, 3.0, 3.1, 3.2, 3.3, 4.0, 4.1",3380,659,228,115,https://commons.apache.org/collections/,https://github.com/apache/commons-collections.git,https://issues.apache.org/jira/rest/api/2/sear...
commons-compress,"1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1...",2755,484,200,172,https://commons.apache.org/compress/,https://github.com/apache/commons-compress.git,https://issues.apache.org/jira/rest/api/2/sear...
commons-configuration,"1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1...",3717,743,314,188,https://commons.apache.org/configuration/,https://github.com/apache/commons-configuratio...,https://issues.apache.org/jira/rest/api/2/sear...


,Release Date,#Files Total,#Files Production
ant-ivy-1.4.1,2007-01-24 18:30:33,327,240
ant-ivy-2.0.0,2009-01-18 22:08:59,481,352
ant-ivy-2.1.0,2009-09-25 22:18:07,487,357
ant-ivy-2.2.0,2010-09-23 21:53:29,498,363
ant-ivy-2.3.0,2013-01-21 20:12:28,604,451
...,...,...,...
wss4j-1.5.0,2006-05-02 18:17:17,335,278
wss4j-1.6.0,2011-04-06 15:48:42,196,135
wss4j-2.0.0,2014-05-02 10:47:39,693,482
wss4j-2.1.0,2015-04-22 14:14:05,704,493


In [12]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... Brown Corpus
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat............ CESS-CAT Treebank
  [

Hit Enter to continue: book_grammars
  [ ] conll2007........... Dependency Treebanks from CoNLL 2007 (Catalan
                           and Basque Subset)
  [ ] crubadan............ Crubadan Corpus
  [ ] dependency_treebank. Dependency Parsed Treebank
  [ ] dolch............... Dolch Word List
  [ ] europarl_raw........ Sample European Parliament Proceedings Parallel
                           Corpus
  [ ] floresta............ Portuguese Treebank
  [ ] framenet_v15........ FrameNet 1.5
  [ ] framenet_v17........ FrameNet 1.7
  [ ] gazetteers.......... Gazeteer Lists
  [ ] genesis............. Genesis Corpus
  [ ] gutenberg........... Project Gutenberg Selections
  [ ] ieer................ NIST IE-ER DATA SAMPLE
  [ ] inaugural........... C-Span Inaugural Address Corpus
  [ ] indian.............. Indian Language POS-Tagged Corpus
  [ ] jeita............... JEITA Public Morphologically Tagged Corpus (in
                           ChaSen format)
  [ ] kimmo............... PC-KIMMO Data F

      Unzipping tokenizers/punkt.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [27]:
import os
import tarfile
import shutil
import git
from xml.etree.ElementTree import Element, SubElement, Comment, tostring, ElementTree
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize 


basefolder = "smartshark/smartshark"
ps = PorterStemmer()
    
def preprocessSimple(text):
    new_word = ""
    words = word_tokenize(text)
    for w in words:
        new_word = new_word + " " + ps.stem(w)
    return new_word

for project in projects:
    cur_releases = []
    if os.path.isdir(basefolder + "/" + project):
        continue
    os.mkdir(basefolder + "/" + project)
    os.mkdir(basefolder + "/" + project + "/bugrepo")
    os.mkdir(basefolder + "/" + project + "/bugrepo/repository")
    os.mkdir(basefolder + "/" + project + "/gitrepo")
    os.mkdir(basefolder + "/" + project + "/sources")
    # clone repo 
    project_id = Project.objects(name=project).get().id
    vcs_system = VCSSystem.objects(project_id=project_id).get()
    its_system = IssueSystem.objects(project_id=project_id).get()
    f = open(basefolder + "/" + project + '/repo.tar.gz', 'wb')
    f.write(vcs_system.repository_file.read())
    f.close()
    tar = tarfile.open(basefolder + "/" + project + '/repo.tar.gz', "r:gz")
    tar.extractall(basefolder + "/" + project + "/gitrepo")
    tar.close()
    files = os.listdir(basefolder + "/" + project + "/gitrepo/" + project)
    #for f in files:
    #    shutil.move(basefolder + "/" + project + "/gitrepo/" + project+ "/" +f, basefolder + "/" + project + "/gitrepo/")
    #os.rmdir(basefolder + "/" + project + "/gitrepo/" + project)
    last_releaseDate = None
    for release in releases[project]:
        os.mkdir(basefolder + "/" + project + "/sources/" + release['version'])
        if 'corrected_revision' in release:
            revision_hash = release['corrected_revision']
        else:
            revision_hash = release['revision_hash']
        release_commit = Commit.objects(revision_hash=revision_hash).get()
        num_bugs = 0
        top = Element('bugrepository')
        for issue in Issue.objects(issue_system_id=its_system.id,issue_type_verified__iexact='bug'):
            num_files = 0
            if jira_is_resolved_and_fixed(issue) and issue.updated_at < release_commit.committer_date and (last_releaseDate == None or issue.updated_at > last_releaseDate):
                num_bugs += 1
                bug = SubElement(top, 'bug', {'id':str(num_bugs), 'fixversion': str(issue.fix_versions), 'opendate': issue.created_at.strftime("%Y-%m-%d %H:%M:%S"), 'fixdate': issue.updated_at.strftime("%Y-%m-%d %H:%M:%S"), "resolution": 'Fixed'})
                fixedFiles = SubElement(bug, 'fixedFiles')
                classNames = []
                for commits in Commit.objects(fixed_issue_ids=issue.id):
                    for file_action in FileAction.objects(commit_id=commits.id):
                        if file_action.mode == "M":
                            for className in CodeEntityState.objects(file_id=file_action.file_id, ce_type='class'):
                                classNames.append(className.long_name)
                classNames = list(dict.fromkeys(classNames))
                for className in classNames:
                    file = SubElement(fixedFiles, 'file', { 'type' : 'M'})
                    file.text = className
                    num_files += 1
                #bug = SubElement(top, 'bug', {'id':str(num_bugs), 'opendate': issue.created_at.strftime("%Y-%m-%d %H:%M:%S"), 'fixdate': issue.updated_at.strftime("%Y-%m-%d %H:%M:%S"), "resolution": 'Fixed'})
                bugInformation = SubElement(bug, 'buginformation')
                summary = SubElement(bugInformation, 'summary')
                if issue.title:
                    summary.text = preprocessSimple(issue.title)
                else:
                    summary.text = ""
                description = SubElement(bugInformation, 'description')
                if issue.desc:
                    description.text = preprocessSimple(issue.desc)
                else: 
                    description.text = ""
                version = SubElement(bugInformation, 'version')
                version.text = release['version']
                
                linkes = SubElement(bug, 'links')
                if(num_files == 0):
                    top.remove(bug)
        print(num_bugs)      
        tree = ElementTree(top)
        tree.write(basefolder + "/" + project + "/bugrepo/repository/" + release['version'] + ".xml")
        # Clone source code
        repo = git.Repo.clone_from(basefolder + "/" + project + "/gitrepo/" + project, basefolder + "/" + project + "/sources/" + release['version'], no_checkout=True)
        repo.git.checkout(revision_hash)
        last_releaseDate = release_commit.committer_date

14
20
13
8
19
9
6
11
5
9
16
16
1
